# Hello Tergite

This is a showcase of connecting to tergite via the `tergite-qiskit-connector`, running a basic circuit, and retrieving the measurement results.

## Install dependencies

This example depends on:

- [qiskit](https://github.com/Qiskit/qiskit)
- [tergite-qiskit-connector](https://test.pypi.org/project/tergite-qiskit-connector/)

To install these dependencies, run the follwing commands in your terminal
```
$ pip install qiskit
$ pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple tergite-qiskit-connector
```

## Import the basic dependencies

In [1]:
import time
import qiskit.circuit as circuit
import qiskit.compiler as compiler
from tergite_qiskit_connector.providers.tergite import Tergite
from tergite_qiskit_connector.providers.tergite.provider_account import ProviderAccount

/Users/skoghm/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Configure Session
Before we get any further, we will take the time to define some of the parameters we will use for our tergite job.

In [2]:
# the Tergite API URL e.g. "https://api.tergite.example"
API_URL = "https://api.qal9000.se"
# API token for connecting to tergite
API_TOKEN = "Yd0_k770B1pvzZdnq3L-oXRLsh2ME-F3HpD2U4DguZ0"
# The name of the Quantum Computer to use from the available quantum computers
BACKEND_NAME = "SimulatorC"
# the name of this service. For your own bookkeeping.
SERVICE_NAME = "local"
# the timeout in seconds for how long to keep checking for results
POLL_TIMEOUT = 300

## Get the Tergite Backend
The backend object can now be obtained. A detailed list of the backend properties — such as the available gate set, coupling map and number of qubits — is availablde by printing the backend object. 

In [3]:
# provider account creation can be skipped in case you already saved
# your provider account to the `~/.qiskit/tergiterc` file.
# See below how that is done.
account = ProviderAccount(service_name=SERVICE_NAME, url=API_URL, token=API_TOKEN)

provider = Tergite.use_provider_account(account)
# to save this account to the `~/.qiskit/tergiterc` file, add the `save=True`
# provider = Tergite.use_provider_account(account, save=True)

# Get the tergite backend in case you skipped provider account creation
# provider = Tergite.get_provider(service_name=SERVICE_NAME)
backend = provider.get_backend(BACKEND_NAME)
backend.set_options(shots=1024)
print(backend)

TergiteBackend object @ 0x11f5ffbb0:
  backend_name:               SimulatorC
  backend_version:            2023.11.15
  n_qubits:                   5
  basis_gates:                []
  gates:                      []
  local:                      False
  simulator:                  False
  conditional:                False
  open_pulse:                 True
  memory:                     False
  max_shots:                  inf
  coupling_map:               [[0, 2], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [4, 2]]
  dynamic_reprate_enabled:    False
  supported_instructions:     [(Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (0,)), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (1,)), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (2,)), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (3,)), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (4,)), (Instruction(name='rx', num_qubits=1, 

## Create the Qiskit Circuit
To test our connection, we will implement a short test circuit. The circuit we will run produces the Bell state $|\Psi\rangle = |00\rangle + |11\rangle.$ 

In [4]:
qc = circuit.QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)

We can visualize and verify our circuit with Qiskit's built in `draw()` method. The output format of `qc.draw()` can be changed, see https://docs.quantum.ibm.com/build/circuit-visualization. Note the two added measurements and corresponding classical bit registers `meas_0` and `meas_1`.

In [5]:
qc.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

To measure the prepared Bell state we add explicit measurements to all qubits using `qc.measure_all()`. This will perform a meaurement in the so-called computational basis, $\langle q_n|Z|q_n\rangle$, mapping the eigenvalues $\{-1,1\}$ to the classical binary values $\{0,1\}$. Drawing the final circuit shows the additional measurement operations and the classical bit register `meas_0` and `meas_1`.

In [6]:
qc.measure_all()
qc.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas_0: ══════════════╩══╬═
                         ║ 
meas_1: ═════════════════╩═

## Compile Circuit
In order to execute the circuit on physical hardware, the circuit needs to be compiled (or transpiled) to the target architecture. At the least, transpilation accounts for the QPU's native gate set and the qubit connectivity on the QPU. Many transpilers also offer some level of optimization, reducing the circuit size.

In [7]:
tc = compiler.transpile(qc, backend=backend)
tc.draw()

This Target object contains multiqubit gates that operate on > 2 qubits. This will not be reflected in the output coupling map.
This Target object contains multiqubit gates that operate on > 2 qubits. This will not be reflected in the output coupling map.
This Target object contains multiqubit gates that operate on > 2 qubits. This will not be reflected in the output coupling map.
This Target object contains multiqubit gates that operate on > 2 qubits. This will not be reflected in the output coupling map.


TranspilerError: "Unable to map source basis {('cx', 2), ('h', 1), ('measure', 1), ('barrier', 2)} to target basis {'reset', 'rz', 'barrier', 'measure', 'snapshot', 'delay', 'rx'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x11808ba00>."

## Run the Circuit
Once the cicruit has been compiled to the native gate set and connectivity, we use it to submit a job to the backend. 

In [ ]:
job = backend.run(tc, meas_level=2, meas_return="single")

## See the Results
When the job has been submitted, we will need to wait potential queue time and time required to execute the job.

In [ ]:
elapsed_time = 0
result = None
while result is None:
    if elapsed_time > POLL_TIMEOUT:
        raise TimeoutError(f"result polling timeout {POLL_TIMEOUT} seconds exceeded")

    time.sleep(1)
    elapsed_time += 1
    result = job.result()

result.get_counts()